In [3]:
from dataset import get_path, get_subjects, epoch_data
from utils import decod, correlate, match_list
import mne_bids
from pathlib import Path
import pandas as pd
import numpy as np
import mne
import spacy

nlp = spacy.load('fr_core_news_sm')

from sklearn.model_selection import KFold, cross_val_predict
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.linear_model import RidgeCV
from wordfreq import zipf_frequency
from Levenshtein import editops
import matplotlib.pyplot as plt
import matplotlib

matplotlib.use("Agg")
mne.set_log_level(False)

In [5]:
report = mne.Report()
path = get_path('LPP_read')
subjects = get_subjects(path)
RUN = 1

print("\nSubjects for which the decoding will be tested: \n")
print(subjects)

for subject in subjects:  # Ignore the first one

    print(f"Subject {subject}'s decoding started")
    epochs = []
    for run_id in range(1, RUN + 1):
        print(".", end="")
        epo = epoch_data(subject, "%.2i" % run_id, task='listen', path=path)
        epo.metadata["label"] = f"run_{run_id}"
        epochs.append(epo)

    # Quick fix for the dev_head: has to be
    # fixed before doing source reconstruction
    for epo in epochs:
        epo.info["dev_head_t"] = epochs[0].info["dev_head_t"]
        # epo.info['nchan'] = epochs[0].info['nchan']

    epochs = mne.concatenate_epochs(epochs)

    # Get the evoked potential averaged on all epochs for each channel
    evo = epochs.average(method="median")
    evo.plot(spatial_colors=True)

    # Handling the data structure
    epochs.metadata["kind"] = epochs.metadata.trial_type.apply(
        lambda s: eval(s)["kind"]
    )
    epochs.metadata["word"] = epochs.metadata.trial_type.apply(
        lambda s: eval(s)["word"]
    )
    epochs.metadata['closing'] = epochs.metadata.closing_.fillna(0)
    # Run a linear regression between MEG signals
    # and word frequency classification
    X = epochs.get_data()


    y = epochs.metadata['closing']

    R_vec = decod(X, y)
    R_vec_avg = np.mean(R_vec,axis = 1)

    fig, ax = plt.subplots(1, figsize=[6, 6])
    dec = plt.fill_between(epochs.times, R_vec_avg)
    # plt.show()
    report.add_evokeds(evo, titles=f"Evoked for sub {subject} ")
    report.add_figure(fig, title=f"decoding for subject {subject}")
    # report.add_figure(dec, subject, tags="word")
    report.save("./figures/reading_decoding_embeddings.html", open_browser=False, overwrite=True)

    print("Finished!")



Subjects for which the decoding will be tested: 

['1', '2', '3', '4', '5', '6']
Subject 1's decoding started
.Running the script on RAW data:
run 01, subject: 1


/home/co/workspace_LPP/code/neurospin-petit-prince/decoding/reading/dataset.py:60: RuntimeWarning: This file contains raw Internal Active Shielding data. It may be distorted. Elekta recommends it be run through MaxFilter to produce reliable results. Consider closing the file and running MaxFilter on the data.
  run=run_id,
/home/co/workspace_LPP/code/neurospin-petit-prince/decoding/reading/dataset.py:60: RuntimeWarning: Omitted 4 annotation(s) that were outside data range.
  run=run_id,
/home/co/workspace_LPP/code/neurospin-petit-prince/decoding/reading/dataset.py:60: RuntimeWarning: The unit for channel(s) STI001, STI002, STI003, STI004, STI005, STI006, STI007, STI008, STI009, STI010, STI011, STI012, STI013, STI014, STI015, STI016, STI101, STI201, STI301 has changed from V to NA.
  run=run_id,
/home/co/workspace_LPP/code/neurospin-petit-prince/decoding/reading/dataset.py:89: RuntimeWarning: The measurement information indicates a low-pass frequency of 20.0 Hz. The decim=20 parameter w

AttributeError: 'DataFrame' object has no attribute 'closing_'

In [7]:
epochs.metadata

,index,Unnamed: 0,word,onset,duration,trial_type,start,label,kind
0,0,0,Lorsque,0.70,0.3,"{'kind': 'word', 'word': 'Lorsque'}",44.534,run_1,word
1,1,1,javais,1.05,0.3,"{'kind': 'word', 'word': 'javais'}",44.889,run_1,word
2,2,2,six,1.40,0.3,"{'kind': 'word', 'word': 'six'}",45.189,run_1,word
3,3,3,ans,1.75,0.3,"{'kind': 'word', 'word': 'ans'}",45.489,run_1,word
4,4,4,jai,2.10,0.3,"{'kind': 'word', 'word': 'jai'}",45.806,run_1,word
...,...,...,...,...,...,...,...,...,...
1406,1459,1459,ne,511.35,0.3,"{'kind': 'word', 'word': 'ne'}",528.061,run_1,word
1407,1460,1460,peut,511.70,0.3,"{'kind': 'word', 'word': 'peut'}",528.378,run_1,word
1408,1461,1461,pas,512.05,0.3,"{'kind': 'word', 'word': 'pas'}",528.694,run_1,word
1409,1462,1462,aller,512.40,0.3,"{'kind': 'word', 'word': 'aller'}",529.011,run_1,word
